# Final Report  - Time Series Project

Tarrence Nichols and Nathan Sharick - Innis Cohort

---

### Project Description

- 

### Project Goals

- 

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from env import get_db_url
from sklearn.model_selection import train_test_split
from acquire import get_superstore_df
from acquire import clean_superstore_df

### Acquire, Clean, and Wrangle the dataset

- All data acquisition and data wrangling functions are located in the accompanying acquire.py file

- The get_superstore_df function does...

- The clean_superstore_df does...



In [ ]:
#acquire the dataset using the get_superstore_df function
df = get_superstore_df()
#complete initial cleaning and wrangling of the dataset with the clean_superstore_df function
df = clean_superstore_df(df)

---

### Exploration of Products

- point 1...

- point 2...

- point 3...

- etc...

### Key Takeaways from Product Exploration

- takeaway 1...

- etc...

---

### Exploration of Regional Customer Data

- point 1...

- etc...

In [ ]:
#acquire the dataset with the get_superstore_df function
df = get_superstore_df()
#complete initial data cleaning/wrangling with the clean_superstore_df function
df = clean_superstore_df(df)
#seperate the dataset by region for regional and customer analysis
east = df[df.region == 'East']
west = df[df.region == 'West']
central = df[df.region == 'Central']
south = df[df.region == 'South']

#### Visualize yearly sales by region

In [ ]:
##Plot the total yearly sales by region
#plot each region as its own line
east_sales.resample('Y').sum().plot()
west_sales.resample('Y').sum().plot()
central_sales.resample('Y').sum().plot()
south_sales.resample('Y').sum().plot()
#add a label to the x_axis
plt.xlabel('Date of Order')
#add a label to the y axis
plt.ylabel('Sales Value')
#add a title to the plot
plt.title('Yearly Total Sales by Region')
#add a legend to the plot
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))

In [ ]:
##Plot the average order value per year by region
#plot each region as its own line
east_sales.resample('Y').mean().plot()
west_sales.resample('Y').mean().plot()
central_sales.resample('Y').mean().plot()
south_sales.resample('Y').mean().plot()
#add a label to the x axis
plt.xlabel('Date of Order')
#add a label to the y axis
plt.ylabel('Sale Value')
#add a title to the plot
plt.title('Yearly Average Customer Order Value by Region')
#add a legend to the plot
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))

#### Use a RFM analysis to rate and group each customer from the central region for targeted marketing 

In [ ]:
##RFM Analysis##
#create a new dataframe with customer name and sales for all orders from the central region
central_cust_sales = central[['cust_name', 'sales']]
#add a column that contains the data of the most recent order made by the customer
central_cust_sales['last_purchase'] = central_cust_sales['cust_name'].apply(lambda x: central_cust_sales[central_cust_sales['cust_name'] == x].index.max())
#set a 'today' variable to calculate how many days since the customers last order
today = pd.to_datetime('2017-12-30') 
#add a column for recency with the number of days since the customers last order
central_cust_sales['recency'] = central_cust_sales['last_purchase'].apply(lambda x: (today - x).days)
#add a column for frequency with the total number of orders the customer has made
central_cust_sales['frequency'] = central_cust_sales['cust_name'].apply(lambda x: central_cust_sales[central_cust_sales['cust_name'] == x].size)
#add a column for monetary with the total sales value for all that customers orders
central_cust_sales['monetary'] = central_cust_sales['cust_name'].apply(lambda x: central_cust_sales[central_cust_sales['cust_name'] == x].sales.sum())
#drop the sales and last purchase columns that were used to calculate the other columns
central_customer_rating = central_cust_sales.drop(columns=['sales', 'last_purchase'])
#group the dataframe by customer name to remove duplicate rows
central_customer_rating = central_customer_rating.groupby('cust_name').max()

#rank the customers in each of the three categories (recency, frequency, monetary)
central_customer_rating['r_rank'] = central_customer_rating['recency'].rank(ascending=False)
central_customer_rating['f_rank'] = central_customer_rating['frequency'].rank(ascending=True)
central_customer_rating['m_rank'] = central_customer_rating['monetary'].rank(ascending=True)
#normalize the rankings for overall scoring
central_customer_rating['r_rank_norm'] = (central_customer_rating['r_rank']/central_customer_rating['r_rank'].max())*100
central_customer_rating['f_rank_norm'] = (central_customer_rating['f_rank']/central_customer_rating['f_rank'].max())*100
central_customer_rating['m_rank_norm'] = (central_customer_rating['m_rank']/central_customer_rating['m_rank'].max())*100
#drop the original rank columns used for calculations
central_customer_rating.drop(columns=['r_rank', 'f_rank', 'm_rank'], inplace=True)

#calculate the customers rfm score
central_customer_rating['rfm_score'] = (0.15*central_customer_rating['r_rank_norm'])+(0.28*central_customer_rating['f_rank_norm'])+(0.57*central_customer_rating['m_rank_norm'])
#change the rfm score to a score between 0 and 5
central_customer_rating['rfm_score'] *= 0.05
#round the customers rfm score to 2 decimal places
central_customer_rating.rfm_score = central_customer_rating.rfm_score.round(2)

#add a column that groups customers into overall rating groups
central_customer_rating['overall_customer_rating'] = pd.cut(central_customer_rating.rfm_score, bins=[0, 1.6, 3, 4, 4.5, 5], labels=['lost customer', 'low value customer', 'moderate value customer', 'high value customer', 'top customer'])
#add a column that groups customers by recency ranking
central_customer_rating['recency_rating'] = pd.cut(central_customer_rating.recency, bins=[0, 180, 360, 1500 ], labels=['high', 'moderate', 'low'])
#add a column that groups customers by frequency rating
central_customer_rating['frequency_rating'] = pd.cut(central_customer_rating.frequency, bins=[0, 13, 23, 33], labels=['low', 'moderate', 'high'])
#add a column that groups customers by monetary rating
central_customer_rating['monetary_rating'] = pd.cut(central_customer_rating.monetary, bins=[0, 1000, 5000, 11000], labels=['low', 'moderate', 'high'])
#drop the normalized rank columns used to calculate customer groupings
central_customer_rating.drop(columns=['r_rank_norm', 'f_rank_norm', 'm_rank_norm'], inplace=True)
#view the resulting dataframe
central_customer_rating


### Key Takeaways from Customer Exploration

- Takeaway 1...

- etc...

---

### Summary

- point 1...

- etc...


### Marketing Recomendations

- **Recomendations based on product research**

    - Recomendation 1....
    
    - Recomendation 2...
    
- **Recomendations based on customer research**

    - Recomendation 1...
    
    - Recomendation 2...

### Next Steps

- next step 1...

- etc...